## Load the data

In [29]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
n_samples=50
data_samples = dataset.data[:n_samples]

n_features=300
tmp = CountVectorizer(max_features=n_features,
                                stop_words='english')
corpus = tmp.fit_transform(data_samples)
corpus.toarray().shape


(50, 300)

## LDA model

In [60]:
import numpy as np
import time
import tqdm

class myLDA:
    
    def __init__(self, corpus, n_topics):
        '''
            Parameters
            ----------
            corpus: text set
            n_topics: number of topic
        '''
        
        self.Doc = corpus.toarray() # Documents
        self.W = np.arange(self.Doc.shape[1]) # Words
        self.n_doc = self.Doc.shape[0]  # num of docs
        self.n_words = self.W.shape[0]  # num of words
        self.n_topics = n_topics  # num of topics
        # Dirichlet priors
        self.alpha = np.ones(self.n_topics)
        self.eta = np.ones(self.n_words)
        # Z: word topic assignment  Pi: document topic distribution B: word topic distribution
        self.Z = np.zeros(shape=[self.n_doc, self.n_words])
        self.Pi = np.zeros([self.n_doc, self.n_topics])
        self.B = np.zeros([self.n_topics, self.n_words])
        self.init_parametres()

    def init_parametres(self):
        '''
            Initialize Z， Pi, B
        '''
        for i in range(self.n_doc):
            for j in range(self.n_words):
                self.Z[i, j] = np.random.randint(self.n_topics)
        for i in range(self.n_doc):
            self.Pi[i] = np.random.dirichlet(self.alpha)
        for i in range(self.n_topics):
            self.B[i] = np.random.dirichlet(self.eta)
            
    def update(self, method="Gibbs_Sampling", epoch=1, w=1, K=1):
        '''
            Update Z， Pi, B according to the article "On Smoothing and Inference for Topic Models"
            The link is 'https://arxiv.org/pdf/1205.2662.pdf'
        '''
        start = time.time()
        for it in tqdm.tqdm(range(epoch)):
            eval("self." + method)(w,K)
        end = time.time()
        print("Time of training:",end-start)
        
    def ML(self, w=1, K=1):
        '''
            ML ESTIMATION
        '''
        for i in range(self.n_doc):
            for v in range(self.n_words):
                p_iv = np.exp(np.log(self.Pi[i]) + np.log(self.B[:, self.Doc[i, v]]))
                p_iv/= np.sum(p_iv)
                self.Z[i, v] = p_iv.argmax() #update word topic assignment 
        for i in range(self.n_doc):
            m = np.zeros(self.n_topics)
            for k in range(self.n_topics):
                m[k] = np.sum(self.Z[i] == k)
            self.Pi[i, :] = m / sum(m) #update document topic distribution 
        for k in range(self.n_topics):
            n = np.zeros(self.n_words)
            for v in range(self.n_words):
                for i in range(self.n_doc):
                    for l in range(self.n_words):
                        n[v] += (self.Doc[i, l] == v) and (self.Z[i, l] == k)
            self.B[k, :] = n / sum(n) #update word topic distribution

    def MAP(self, w=1, K=1):
        '''
            MAP ESTIMATION
        '''
        for i in range(self.n_doc):
            m = np.zeros(self.n_topics)
            for k in range(self.n_topics):
                m[k] = np.sum(Z[i] == k)
            self.Pi[i, :] = (m + self.alpha-1) / (sum(m) + K*self.alpha - K) #update document topic distribution  

        for k in range(self.n_topics):
            n = np.zeros(self.n_words)
            for v in range(self.n_words):
                for i in range(self.n_doc):
                    for l in range(self.n_words):
                        n[v] += (self.Doc[i, l] == v) and (self.Z[i, l] == k)
            self.B[k, :] = (n + self.eta-1) / (sum(n) + w*self.eta - w) #update word topic distribution
        for i in range(n_doc):
            m = np.zeros(self.n_topics)
            for k in range(self.n_topics):
                m[k] = np.sum(self.Z[i] == k)
            for v in range(self.n_words):
                p_iv = Pi[i] * self.B[:, self.Doc[i, v]] * (sum(m) + K*self.alpha - K)
                p_iv/= np.sum(p_iv)
                self.Z[i, v] = p_iv.argmax() #update word topic assignment
    
    def VB(self, w=1, K=1):
        '''
            VARIATIONAL BAYES
        '''
        for i in range(self.n_doc):
            m = np.zeros(self.n_topics)
            for k in range(self.n_topics):
                m[k] = np.sum(Z[i] == k)
            self.Pi[i, :] = (m + self.alpha - 0.5) / (sum(m) + K*self.alpha - 0.5) #update document topic distribution  

        for k in range(self.n_topics):
            n = np.zeros(self.n_words)
            for v in range(self.n_words):
                for i in range(self.n_doc):
                    for l in range(self.n_words):
                        n[v] += (self.Doc[i, l] == v) and (self.Z[i, l] == k)
            self.B[k, :] = (n + self.eta - 0.5) / (sum(n) + w*self.eta - 0.5) #update word topic distribution
        for i in range(n_doc):
            m = np.zeros(self.n_topics)
            for k in range(self.n_topics):
                m[k] = np.sum(self.Z[i] == k)
            for v in range(self.n_words):
                p_iv = Pi[i] * self.B[:, self.Doc[i, v]] * (sum(m) + K*self.alpha - 0.5)
                p_iv/= np.sum(p_iv)
                self.Z[i, v] = p_iv.argmax() #update word topic assignment   
    
    def CVB0(self, w=1, K=1):
        '''
            COLLAPSED VARIATIONAL BAYES
        '''
        for i in range(self.n_doc):
            m = np.zeros(self.n_topics)
            for k in range(self.n_topics):
                m[k] = np.sum(Z[i] == k)
            self.Pi[i, :] = (m + self.alpha - 0.5) / (sum(m) + K*self.alpha - 0.5) #update document topic distribution  

        for k in range(self.n_topics):
            n = np.zeros(self.n_words)
            for v in range(self.n_words):
                for i in range(self.n_doc):
                    for l in range(self.n_words):
                        n[v] += (self.Doc[i, l] == v) and (self.Z[i, l] == k)
            self.B[k, :] = (n + self.eta - 0.5) / (sum(n) + w*self.eta - 0.5) #update word topic distribution
        for i in range(self.n_doc):
            for v in range(self.n_words):
                p_iv = np.exp(np.log(self.Pi[i]) + np.log(self.B[:, self.Doc[i, v]]))
                p_iv/= np.sum(p_iv)
                self.Z[i, v] = np.random.multinomial(1, p_iv).argmax() #update word topic assignment 
                
    def Gibbs_Sampling(self, w=1, K=1):
        '''
            COLLAPSED GIBBS SAMPLING
        '''
        for i in range(self.n_doc):
            for v in range(self.n_words):
                p_iv = np.exp(np.log(self.Pi[i]) + np.log(self.B[:, self.Doc[i, v]]))
                p_iv/= np.sum(p_iv)
                self.Z[i, v] = np.random.multinomial(1, p_iv).argmax() #update word topic assignment 
        for i in range(self.n_doc):
            m = np.zeros(self.n_topics)
            for k in range(self.n_topics):
                m[k] = np.sum(self.Z[i] == k)
            self.Pi[i, :] = np.random.dirichlet(self.alpha + m) #update document topic distribution 
        for k in range(n_topics):
            n = np.zeros(n_words)
            for v in range(self.n_words):
                for i in range(self.n_doc):
                    for l in range(self.n_words):
                        n[v] += (self.Doc[i, l] == v) and (self.Z[i, l] == k)
            self.B[k, :] = np.random.dirichlet(self.eta + n) #update word topic distribution
    
    def get_w_t(self):
        '''
            Get word topic assignment
        '''
        return self.Z
    
    def get_t_d(self):
        '''
            Get document topic distribution
        '''
        return self.Pi
    
    def get_w_d(self):
        '''
            Get word topic distribution
        '''
        return self.B
    
    def evaluate(self):
        '''
            Perplexity
        '''
        epision = 1
        P = np.log(np.dot(self.Pi, self.B) + epision)
        s = sum(sum(P))
        t = -(s / (self.n_topics * self.n_words))
        return np.exp(t)

## Test the model 

In [61]:
model1 = myLDA(corpus,5)
model1.update(method ="ML", epoch = 10)
print("Document topic distribution:\n",np.round(model1.get_t_d(), decimals=3))
print("Model score:\n",model1.evaluate())





  0%|          | 0/10 [00:00<?, ?it/s]



 10%|█         | 1/10 [00:52<07:51, 52.39s/it]/home/gjx/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in log




 20%|██        | 2/10 [01:51<07:14, 54.35s/it]



 30%|███       | 3/10 [02:58<06:47, 58.20s/it]



 40%|████      | 4/10 [04:20<06:31, 65.28s/it]



 50%|█████     | 5/10 [06:56<07:43, 92.63s/it]



 60%|██████    | 6/10 [07:56<05:31, 82.76s/it]



 70%|███████   | 7/10 [09:03<03:54, 78.00s/it]



 80%|████████  | 8/10 [11:20<03:11, 95.82s/it]



 90%|█████████ | 9/10 [13:19<01:42, 102.59s/it]



100%|██████████| 10/10 [15:19<00:00, 107.78s/it]





Time of training: 919.029703617096
Document topic distribution:
 [[0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    0.    0.    0.    1.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.007 0.    0.    0.    0.993]
 [0.35  0.43  0.    0.22  0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.997 0.    0.    0.    0.003]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [1.    0.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [1.    0.    0.    0.    0.   ]
 [0.013 0.    0.977 0.01  0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.01  0.99  0.    0.    0.   ]
 [0.007 0.9

In [62]:
model2 = myLDA(corpus,5)
model2.update(method ="MAP", epoch = 10)
print("Document topic distribution:\n",np.round(model2.get_t_d(), decimals=3))
print("Model score:\n",model2.evaluate())





  0%|          | 0/10 [00:00<?, ?it/s]



 10%|█         | 1/10 [01:58<17:44, 118.33s/it]



 20%|██        | 2/10 [03:56<15:46, 118.32s/it]



 30%|███       | 3/10 [05:55<13:49, 118.56s/it]



 40%|████      | 4/10 [06:54<10:03, 100.53s/it]



 50%|█████     | 5/10 [07:50<07:15, 87.13s/it] 



 60%|██████    | 6/10 [08:41<05:05, 76.44s/it]



 70%|███████   | 7/10 [09:32<03:26, 68.92s/it]



 80%|████████  | 8/10 [10:26<02:08, 64.21s/it]



 90%|█████████ | 9/10 [11:18<01:00, 60.57s/it]



100%|██████████| 10/10 [12:09<00:00, 57.79s/it]





Time of training: 729.5461647510529
Document topic distribution:
 [[0.193 0.243 0.173 0.193 0.197]
 [0.253 0.183 0.193 0.223 0.147]
 [0.183 0.183 0.167 0.257 0.21 ]
 [0.21  0.21  0.18  0.197 0.203]
 [0.193 0.197 0.247 0.167 0.197]
 [0.24  0.18  0.177 0.217 0.187]
 [0.233 0.213 0.193 0.183 0.177]
 [0.213 0.2   0.22  0.203 0.163]
 [0.213 0.18  0.213 0.223 0.17 ]
 [0.177 0.203 0.2   0.187 0.233]
 [0.263 0.18  0.207 0.18  0.17 ]
 [0.227 0.243 0.203 0.19  0.137]
 [0.193 0.18  0.2   0.197 0.23 ]
 [0.18  0.177 0.28  0.18  0.183]
 [0.243 0.17  0.15  0.207 0.23 ]
 [0.193 0.163 0.227 0.187 0.23 ]
 [0.177 0.16  0.24  0.227 0.197]
 [0.24  0.19  0.203 0.197 0.17 ]
 [0.223 0.193 0.213 0.173 0.197]
 [0.153 0.217 0.2   0.197 0.233]
 [0.233 0.223 0.177 0.177 0.19 ]
 [0.177 0.24  0.2   0.197 0.187]
 [0.233 0.177 0.167 0.223 0.2  ]
 [0.207 0.19  0.197 0.187 0.22 ]
 [0.207 0.2   0.227 0.213 0.153]
 [0.187 0.17  0.227 0.23  0.187]
 [0.18  0.237 0.22  0.193 0.17 ]
 [0.213 0.183 0.183 0.197 0.223]
 [0.2   0.

In [63]:
model3 = myLDA(corpus,5)
model3.update(method ="VB", epoch = 10)
print("Document topic distribution:\n",np.round(model3.get_t_d(), decimals=3))
print("Model score:\n",model3.evaluate())





  0%|          | 0/10 [00:00<?, ?it/s]



 10%|█         | 1/10 [00:57<08:39, 57.77s/it]



 20%|██        | 2/10 [01:52<07:34, 56.83s/it]



 30%|███       | 3/10 [02:44<06:27, 55.33s/it]



 40%|████      | 4/10 [03:36<05:27, 54.52s/it]



 50%|█████     | 5/10 [04:28<04:27, 53.59s/it]



 60%|██████    | 6/10 [05:21<03:33, 53.38s/it]



 70%|███████   | 7/10 [06:17<02:42, 54.16s/it]



 80%|████████  | 8/10 [07:12<01:48, 54.50s/it]



 90%|█████████ | 9/10 [08:08<00:54, 54.82s/it]



100%|██████████| 10/10 [09:05<00:00, 55.72s/it]





Time of training: 545.8619277477264
Document topic distribution:
 [[0.195 0.245 0.175 0.195 0.198]
 [0.255 0.185 0.195 0.225 0.148]
 [0.185 0.185 0.168 0.258 0.211]
 [0.211 0.211 0.181 0.198 0.205]
 [0.195 0.198 0.248 0.168 0.198]
 [0.241 0.181 0.178 0.218 0.188]
 [0.235 0.215 0.195 0.185 0.178]
 [0.215 0.201 0.221 0.205 0.165]
 [0.215 0.181 0.215 0.225 0.171]
 [0.178 0.205 0.201 0.188 0.235]
 [0.265 0.181 0.208 0.181 0.171]
 [0.228 0.245 0.205 0.191 0.138]
 [0.195 0.181 0.201 0.198 0.231]
 [0.181 0.178 0.281 0.181 0.185]
 [0.245 0.171 0.151 0.208 0.231]
 [0.195 0.165 0.228 0.188 0.231]
 [0.178 0.161 0.241 0.228 0.198]
 [0.241 0.191 0.205 0.198 0.171]
 [0.225 0.195 0.215 0.175 0.198]
 [0.155 0.218 0.201 0.198 0.235]
 [0.235 0.225 0.178 0.178 0.191]
 [0.178 0.241 0.201 0.198 0.188]
 [0.235 0.178 0.168 0.225 0.201]
 [0.208 0.191 0.198 0.188 0.221]
 [0.208 0.201 0.228 0.215 0.155]
 [0.188 0.171 0.228 0.231 0.188]
 [0.181 0.238 0.221 0.195 0.171]
 [0.215 0.185 0.185 0.198 0.225]
 [0.201 0.

In [64]:
model4 = myLDA(corpus,5)
model4.update(method ="CVB0", epoch = 10)
print("Document topic distribution:\n",np.round(model4.get_t_d(), decimals=3))
print("Model score:\n",model4.evaluate())





  0%|          | 0/10 [00:00<?, ?it/s]



 10%|█         | 1/10 [00:53<07:58, 53.11s/it]



 20%|██        | 2/10 [01:45<07:02, 52.85s/it]



 30%|███       | 3/10 [02:38<06:10, 52.92s/it]



 40%|████      | 4/10 [03:30<05:16, 52.80s/it]



 50%|█████     | 5/10 [04:22<04:21, 52.35s/it]



 60%|██████    | 6/10 [05:13<03:28, 52.12s/it]



 70%|███████   | 7/10 [06:08<02:38, 52.82s/it]



 80%|████████  | 8/10 [07:07<01:49, 54.66s/it]



 90%|█████████ | 9/10 [08:04<00:55, 55.44s/it]



100%|██████████| 10/10 [08:55<00:00, 54.09s/it]





Time of training: 535.4612009525299
Document topic distribution:
 [[0.195 0.245 0.175 0.195 0.198]
 [0.255 0.185 0.195 0.225 0.148]
 [0.185 0.185 0.168 0.258 0.211]
 [0.211 0.211 0.181 0.198 0.205]
 [0.195 0.198 0.248 0.168 0.198]
 [0.241 0.181 0.178 0.218 0.188]
 [0.235 0.215 0.195 0.185 0.178]
 [0.215 0.201 0.221 0.205 0.165]
 [0.215 0.181 0.215 0.225 0.171]
 [0.178 0.205 0.201 0.188 0.235]
 [0.265 0.181 0.208 0.181 0.171]
 [0.228 0.245 0.205 0.191 0.138]
 [0.195 0.181 0.201 0.198 0.231]
 [0.181 0.178 0.281 0.181 0.185]
 [0.245 0.171 0.151 0.208 0.231]
 [0.195 0.165 0.228 0.188 0.231]
 [0.178 0.161 0.241 0.228 0.198]
 [0.241 0.191 0.205 0.198 0.171]
 [0.225 0.195 0.215 0.175 0.198]
 [0.155 0.218 0.201 0.198 0.235]
 [0.235 0.225 0.178 0.178 0.191]
 [0.178 0.241 0.201 0.198 0.188]
 [0.235 0.178 0.168 0.225 0.201]
 [0.208 0.191 0.198 0.188 0.221]
 [0.208 0.201 0.228 0.215 0.155]
 [0.188 0.171 0.228 0.231 0.188]
 [0.181 0.238 0.221 0.195 0.171]
 [0.215 0.185 0.185 0.198 0.225]
 [0.201 0.

In [65]:
model5 = myLDA(corpus,5)
model5.update(method ="Gibbs_Sampling", epoch = 10)
print("Document topic distribution:\n",np.round(model5.get_t_d(), decimals=3))
print("Model score:\n",model5.evaluate())





  0%|          | 0/10 [00:00<?, ?it/s]



 10%|█         | 1/10 [00:53<07:59, 53.24s/it]



 20%|██        | 2/10 [01:44<07:01, 52.74s/it]



 30%|███       | 3/10 [02:42<06:18, 54.08s/it]



 40%|████      | 4/10 [03:34<05:21, 53.50s/it]



 50%|█████     | 5/10 [04:29<04:30, 54.01s/it]



 60%|██████    | 6/10 [05:23<03:36, 54.09s/it]



 70%|███████   | 7/10 [06:16<02:41, 53.72s/it]



 80%|████████  | 8/10 [07:09<01:47, 53.60s/it]



 90%|█████████ | 9/10 [08:03<00:53, 53.50s/it]



100%|██████████| 10/10 [08:53<00:00, 52.72s/it]





Time of training: 533.9834718704224
Document topic distribution:
 [[0.236 0.755 0.007 0.    0.002]
 [0.171 0.81  0.008 0.003 0.008]
 [0.012 0.978 0.003 0.003 0.004]
 [0.476 0.5   0.003 0.007 0.015]
 [0.224 0.764 0.007 0.002 0.003]
 [0.457 0.532 0.008 0.002 0.001]
 [0.111 0.884 0.001 0.003 0.   ]
 [0.368 0.624 0.002 0.001 0.005]
 [0.224 0.771 0.003 0.002 0.001]
 [0.601 0.379 0.003 0.001 0.017]
 [0.118 0.028 0.849 0.003 0.003]
 [0.21  0.777 0.003 0.006 0.003]
 [0.139 0.851 0.001 0.004 0.005]
 [0.271 0.712 0.005 0.    0.011]
 [0.644 0.348 0.002 0.005 0.   ]
 [0.051 0.938 0.002 0.005 0.004]
 [0.006 0.988 0.003 0.001 0.001]
 [0.633 0.331 0.026 0.006 0.004]
 [0.183 0.786 0.014 0.002 0.014]
 [0.34  0.646 0.011 0.003 0.   ]
 [0.902 0.083 0.004 0.005 0.006]
 [0.21  0.769 0.001 0.006 0.014]
 [0.016 0.965 0.014 0.002 0.003]
 [0.004 0.971 0.007 0.004 0.014]
 [0.29  0.696 0.01  0.001 0.003]
 [0.439 0.552 0.004 0.004 0.002]
 [0.281 0.713 0.002 0.    0.004]
 [0.012 0.971 0.014 0.003 0.   ]
 [0.247 0.